In [2]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy.spatial import distance_matrix

In [39]:
def L_num(n):
    t = np.zeros([np.int(n*(n-1)/2),2])
    count = -1
    for i in range(n-1):
        for j in range((i+1),n,1):
            count = count + 1
            t[count,0] = i; t[count,1] = j
    return(t)

In [197]:
def elk(n,p):
    count = -1
    el1 = el2 = np.zeros([n,np.int(n*(n-1)/2)])
    el1 = np.r_[np.eye(1),np.zeros([n-1,1])]
    for i in range(2,n,1):
        el1 = np.c_[el1,np.r_[np.eye(i),np.zeros([n-i,i])]]
    
    count = -1
    for i in range(1,n,1):
        temp = np.zeros([n,i])
        temp[i,:] = 1
        el2[:,(count+1):(count+i+1)] = temp
        count = count+i
        
    count = -1
    ek1 = ek2 = np.zeros([p,np.int(p*(p-1)/2)])
    ek1 = np.r_[np.eye(1),np.zeros([p-1,1])]
    for i in range(2,p,1):
        ek1 = np.c_[ek1,np.r_[np.eye(i),np.zeros([p-i,i])]]

    count = -1
    for i in range(1,p,1):
        temp = np.zeros([p,i])
        temp[i,:] = 1
        ek2[:,(count+1):(count+i+1)] = temp
        count = count+i
    return(el1,el2,ek1,ek2)

In [883]:
def pdist(xs):
    """Pairwise Euclidean distacne."""
    return np.sqrt(np.sum((xs[:, None] - xs[None, :])**2, -1))

def dist_weight(X,phi):
    dist_X = np.tril(pdist(X))[1:,:-1]
    dist_X = [dist_X[j,i] for  i in range(dist_X.shape[0]) for j in range(i,dist_X.shape[0],1)]
    res = np.exp(- phi * np.array(dist_X)**2)
    return(res)

In [286]:
def prox(v,sigma):
    return(np.max([0,(1-sigma/np.sum(v**2))]) * v)
def tri2vec(i,j,n):
    return(n*(i-1) - i*(i-1)/2 + j -i)

In [875]:
[int(i)-1 for i in neighbors]

[0, 1, 2]

In [902]:
i = n
neighbors = [tri2vec(j,i,n) for j in range(1,i)]  
neighbors

[3.0, 5.0, 6.0]

In [906]:
i = 1
neighbors = [tri2vec(i,j,n) for j in range((i+1),(n+1))]
temp1 = w[[int(i)-1 for i in neighbors]]
temp2 = list(-np.sort(-temp1))
keep = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
for i in range(2,n,1):
    group_A =  [tri2vec(i,j,n) for j in range((i+1),(n+1))]
    group_B =  [tri2vec(j,i,n) for j in range(1,i)]
    neighbors = group_A
    neighbors.extend(group_B)
    temp1 = w[[int(i)-1 for i in neighbors]]
    temp2 = list(-np.sort(-temp1))
    knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
    keep = list(set(knn).union(set(keep)))
    #print(keep)
i = n
neighbors = [tri2vec(j,i,n) for j in range(1,i)]  
temp1 = w[[int(i)-1 for i in neighbors]]
temp2 = list(-np.sort(-temp1))
knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
keep = list(set(knn).union(set(keep)))
#w[np.array(list(set(list(range(1,len(w)+1))) - set(keep)))-1] = 0
print(keep)

[1.0, 4.0, 6.0]


In [910]:
def knn_weights(w,k,n): # w input array
    i = 1
    neighbors = [tri2vec(i,j,n) for j in range((i+1),(n+1))]
    temp1 = w[[int(i)-1 for i in neighbors]]
    temp2 = list(-np.sort(-temp1))
    keep = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
    for i in range(2,n,1):
        group_A =  [tri2vec(i,j,n) for j in range((i+1),(n+1))]
        group_B =  [tri2vec(j,i,n) for j in range(1,i)]
        neighbors = group_A
        neighbors.extend(group_B)
        temp1 = w[[int(i)-1 for i in neighbors]]
        temp2 = list(-np.sort(-temp1))
        knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
        keep = list(set(knn).union(set(keep)))
    i = n
    neighbors = [tri2vec(j,i,n) for j in range(1,i)]  
    temp1 = w[[int(i)-1 for i in neighbors]]
    temp2 = list(-np.sort(-temp1))
    knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
    keep = list(set(knn).union(set(keep)))
    w[np.array(list(set(list(range(1,len(w)+1))) - set(keep)))-1] = 0
    return(w)

In [363]:
def update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p,el1,el2,ek1,ek2):
    En = np.diag(list(range(n))) + np.diag(list(range(n-1,-1,-1))) - np.ones([n,n]) + np.eye(n) 
    Ep = np.diag(list(range(p))) + np.diag(list(range(p-1,-1,-1))) - np.ones([p,p]) + np.eye(p)
    
    M = np.eye(n) + nu1 * En
    N = nu2 * Ep
    
    lv = lambda_1+ nu1 * v
    lz = lambda_2 + nu2 * z
    C2 = 0 -np.dot((el2-el1),lv.T)
    C3 = np.dot(lz,(ek1-ek2).T)
    C = X +  C2 + C3  
   
    A = linalg.solve_sylvester(M, N.T, C)

    return(A)

In [782]:
def update_vz(lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2):
    
    sigma_1 = gamma_1 * w_l/nu1
    vtemp = al1 - al2 - 1/nu1 * lambda_1
    
    temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
    temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 
    
    v = temp2
    
    sigma_2 = gamma_2 * u_k/nu2
    ztemp = ak1 - ak2 - 1/nu2 * lambda_2
    
    temp3 = np.where((1 - sigma_2/np.sum(ztemp**2,axis=0) < 0), 0 ,1 - sigma_2/np.sum(ztemp**2,axis=0))
    temp4 = np.repeat(temp3,ztemp.shape[0]).reshape(ztemp.shape[1],ztemp.shape[0]).T * ztemp 
  
    z = temp4
    return(v,z)

In [952]:
np.sum(vtemp**2,axis=0) 

array([0.12, 0.  , 0.  , 1.08, 0.  , 0.12])

In [949]:
(1 - sigma_1/np.sum(vtemp**2,axis=0) < 0)

/Users/yaolanqiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/Users/yaolanqiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


array([ True, False, False,  True, False,  True])

In [944]:
    
temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 
    
v = temp2


array([[-0.2,  0. ,  0. , -0.6,  0. , -0.2],
       [-0.2,  0. ,  0. , -0.6,  0. , -0.2],
       [-0.2,  0. ,  0. , -0.6,  0. , -0.2]])

In [623]:
def update_lambda(lambda_1,lambda_2,v, z, nu1,nu2, al1, al2, ak1, ak2):
    lambda_1 = lambda_1 + nu1 * (v - al1 + al2)
    lambda_2 = lambda_2 + nu2 * (z - ak1 + ak2)
    return(lambda_1,lambda_2)

In [700]:
n=X.shape[0]; p=X.shape[1]

tol = 2

eplison_p=L_num(p)
eplison_n=L_num(n)

elks = elk(n,p)
el1,el2,ek1,ek2 = elks[0],elks[1],elks[2],elks[3]

w = np.array(dist_weight(X / np.sqrt(p),phi = 0.5))
w_l = knn_weights(w,1,n)
u = np.array(dist_weight(X.T / np.sqrt(n),phi = 0.5))
u_k = knn_weights(u,1,p)

A = np.zeros([A.shape[0],A.shape[1]])
v= np.zeros([p,eplison_n.shape[0]])
z = np.zeros([n,eplison_p.shape[0]]) 
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

for iters in range(niters):
    print('iter')
    print(iters)
    
    A_old, v_old, z_old, lambda_1_old, lambda_2_old = A, v, z, lambda_1, lambda_2
    # update A
    En = np.diag(list(range(n))) + np.diag(list(range(n-1,-1,-1))) - np.ones([n,n]) + np.eye(n) 
    Ep = np.diag(list(range(p))) + np.diag(list(range(p-1,-1,-1))) - np.ones([p,p]) + np.eye(p)
    
    M = np.eye(n) + nu1 * En
    N = nu2 * Ep
    
    lv = lambda_1+ nu1 * v
    lz = lambda_2 + nu2 * z
    C2 = 0 -np.dot((el2-el1),lv.T)
    C3 = np.dot(lz,(ek1-ek2).T)
    C = X +  C2 + C3  
   
    A = linalg.solve_sylvester(M, N.T, C)
    
    al1 = np.dot(A.T,el1); al2 = np.dot(A.T,el2)
    ak1 = np.dot(A,ek1); ak2 = np.dot(A,ek2)
    
    # update vz
    sigma_1 = gamma_1 * w_l/nu1
    vtemp = al1 - al2 - 1/nu1 * lambda_1
    
    temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
    temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 
    
    v = temp2
    
    sigma_2 = gamma_2 * u_k/nu2
    ztemp = ak1 - ak2 - 1/nu2 * lambda_2
    
    temp3 = np.where((1 - sigma_2/np.sum(ztemp**2,axis=0) < 0), 0 ,1 - sigma_2/np.sum(ztemp**2,axis=0))
    temp4 = np.repeat(temp3,ztemp.shape[0]).reshape(ztemp.shape[1],ztemp.shape[0]).T * ztemp 
  
    z = temp4
 
    # updatae lambda
    lambda_1 = lambda_1 + nu1 * (v - al1 + al2)
    lambda_2 = lambda_2 + nu2 * (z - ak1 + ak2)
    
    print(('A',np.sum(abs(A - A_old))))
    print(('v',np.sum(abs(v - v_old))))
    print(('z',np.sum(abs(z -z_old))))
    print(('l',np.sum(abs(lambda_1 - lambda_1_old))))
    print(('2',np.sum(abs(lambda_2 - lambda_2_old))))
    
    # whether coverage    
    
    if (np.sum(np.abs(A - A_old)) < tol and
        np.sum(np.abs(v - v_old)) < tol and 
        np.sum(np.abs(z - z_old)) < tol and
        np.sum(np.abs(lambda_1 - lambda_1_old)) < tol and
        np.sum(np.abs(lambda_2 - lambda_2_old)) < tol):
        
        print('done')
        break
    



iter
0
('A', 45.00000000000002)
('v', 8.822256055388133)
('z', 3.0000000000000053)
('l', 0.17774394461187448)
('2', 0.0)
iter
1
('A', 3.6611280276940663)
('v', 6.827309413075601)
('z', 2.2500000000000044)
('l', 0.07730941307559647)
('2', 0.0)
iter
2
('A', 2.8289367134613244)
('v', 5.041842551276803)
('z', 1.6875000000000053)
('l', 0.02377853742975322)
('2', 0.0)
iter
3
('A', 2.1142644505419708)
('v', 3.794575949919669)
('z', 1.2656250000000027)
('l', 0.01170058626548709)
('2', 0.0)
iter
4
('A', 1.5856039966817799)
('v', 2.849798954032283)
('z', 0.9492187500000067)
('l', 0.006697796934211908)
('2', 0.0)
iter
5
('A', 1.1896267494120696)
('v', 2.1394421160072636)
('z', 0.7119140624999996)
('l', 0.004191101617334603)
('2', 0.0)
iter
6
('A', 0.8926411636341696)
('v', 1.605884684791484)
('z', 0.533935546875)
('l', 0.0027719747654808558)
('2', 0.0)
done


In [711]:
import random
np.random.choice(mu, 1, replace=False)

array([-0.5])

In [769]:
random.seed(10)
p,n = 200,200
mu = list(np.arange(-6,6,0.5))
theta = 1.5
x = np.zeros([n,p])
row_group = 2
col_group = 8

mu_rc = np.zeros([row_group,col_group])
for r in range(row_group):
    for c in range(col_group):
        mu_rc[r,c] = np.random.choice(mu, 1, replace=False)

row_assign = []
col_assign = []


for i in range(n):
    row_assign.extend(np.random.choice(list(range(row_group)), 1, replace=False))
                            
for i in range(p):
    col_assign.extend(np.random.choice(list(range(col_group)), 1, replace=False))

for i in range(n):
    for j in range(p):
        r = row_assign[i]
        c = col_assign[j]
        mu_temp = mu_rc[r,c]
        x[i,j] = np.random.normal(mu_temp,theta,1)
        

In [770]:
np.sum(np.abs(X))

X = x/100

In [771]:
n=X.shape[0]; p=X.shape[1]

tol = 2

eplison_p=L_num(p)
eplison_n=L_num(n)

elks = elk(n,p)
el1,el2,ek1,ek2 = elks[0],elks[1],elks[2],elks[3]

w = np.array(dist_weight(X / np.sqrt(p),phi = 0.5))
w_l = knn_weights(w,1,n)
u = np.array(dist_weight(X.T / np.sqrt(n),phi = 0.5))
u_k = knn_weights(u,1,p)

A = np.zeros([X.shape[0],X.shape[1]])
v= np.zeros([p,eplison_n.shape[0]])
z = np.zeros([n,eplison_p.shape[0]]) 
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

In [764]:
    al1 = np.dot(A.T,el1); al2 = np.dot(A.T,el2)
    ak1 = np.dot(A,ek1); ak2 = np.dot(A,ek2)

((100, 100), (100, 100))

In [772]:
niters = 1
tol = 1

In [775]:
import time
start_time = time.time()
#print("--- %s seconds ---" % ( - start_time))

for iters in range(niters):

    A_old, v_old, z_old, lambda_1_old, lambda_2_old = A, v, z, lambda_1, lambda_2
    # update A
    En = np.diag(list(range(n))) + np.diag(list(range(n-1,-1,-1))) - np.ones([n,n]) + np.eye(n) 
    Ep = np.diag(list(range(p))) + np.diag(list(range(p-1,-1,-1))) - np.ones([p,p]) + np.eye(p)
    
    M = np.eye(n) + nu1 * En
    N = nu2 * Ep
    
    lv = lambda_1+ nu1 * v
    lz = lambda_2 + nu2 * z
    C2 = 0 -np.dot((el2-el1),lv.T)
    C3 = np.dot(lz,(ek1-ek2).T)
    C = X +  C2 + C3  
   
    A = linalg.solve_sylvester(M, N.T, C)
    
    al1 = np.dot(A.T,el1); al2 = np.dot(A.T,el2)
    ak1 = np.dot(A,ek1); ak2 = np.dot(A,ek2)
    
    # update vz
    sigma_1 = gamma_1 * w_l/nu1
    vtemp = al1 - al2 - 1/nu1 * lambda_1
    
    temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
    temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 
    
    v = temp2
    
    sigma_2 = gamma_2 * u_k/nu2
    ztemp = ak1 - ak2 - 1/nu2 * lambda_2
    
    temp3 = np.where((1 - sigma_2/np.sum(ztemp**2,axis=0) < 0), 0 ,1 - sigma_2/np.sum(ztemp**2,axis=0))
    temp4 = np.repeat(temp3,ztemp.shape[0]).reshape(ztemp.shape[1],ztemp.shape[0]).T * ztemp 
  
    z = temp4
 
    # updatae lambda
    lambda_1 = lambda_1 + nu1 * (v - al1 + al2)
    lambda_2 = lambda_2 + nu2 * (z - ak1 + ak2)
    
    if iters % 10 ==1:
        print('iter')
        print(iters)
    
        print(('A',np.sum(abs(A - A_old))))
        print(('v',np.sum(abs(v - v_old))))
        print(('z',np.sum(abs(z -z_old))))
        print(('l',np.sum(abs(lambda_1 - lambda_1_old))))
        print(('2',np.sum(abs(lambda_2 - lambda_2_old))))
    
    # whether coverage    
    
    if (np.sum(np.abs(A - A_old)) < tol and
        np.sum(np.abs(v - v_old)) < tol and 
        np.sum(np.abs(z - z_old)) < tol and
        np.sum(np.abs(lambda_1 - lambda_1_old)) < tol and
        np.sum(np.abs(lambda_2 - lambda_2_old)) < tol):
        
        print('done')
        break
    
end_time = time.time()
end_time -start_time

0.6992652416229248

In [774]:
#end_time = time.time()
end_time -start_time

0.5415799617767334

In [777]:
X = np.array([[1,5,9],[2,6,10],[3,7,11],[4,8,12]])
A = X
X

array([[ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11],
       [ 4,  8, 12]])

In [778]:
n,p=X.shape[0],X.shape[1]
eplison_p=L_num(p)
eplison_n=L_num(n)

In [808]:
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

In [809]:
A

array([[5.2, 6.2, 7.2],
       [5.4, 6.4, 7.4],
       [5.6, 6.6, 7.6],
       [5.8, 6.8, 7.8]])

In [779]:
elks = elk(n,p)
el1,el2,ek1,ek2 = elks[0],elks[1],elks[2],elks[3]
A = np.zeros([X.shape[0],X.shape[1]])
v= np.zeros([p,eplison_n.shape[0]])
z = np.zeros([n,eplison_p.shape[0]]) 
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

In [793]:

A = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p,el1,el2,ek1,ek2)

In [794]:
al1 = np.dot(A.T,el1); al2 = np.dot(A.T,el2)
ak1 = np.dot(A,ek1); ak2 = np.dot(A,ek2)

In [801]:
al1

array([[5.2, 5.2, 5.4, 5.2, 5.4, 5.6],
       [6.2, 6.2, 6.4, 6.2, 6.4, 6.6],
       [7.2, 7.2, 7.4, 7.2, 7.4, 7.6]])

In [802]:
al2

array([[5.4, 5.6, 5.6, 5.8, 5.8, 5.8],
       [6.4, 6.6, 6.6, 6.8, 6.8, 6.8],
       [7.4, 7.6, 7.6, 7.8, 7.8, 7.8]])

In [803]:
ak1

array([[5.2, 5.2, 6.2],
       [5.4, 5.4, 6.4],
       [5.6, 5.6, 6.6],
       [5.8, 5.8, 6.8]])

In [804]:
ak2

array([[6.2, 7.2, 7.2],
       [6.4, 7.4, 7.4],
       [6.6, 7.6, 7.6],
       [6.8, 7.8, 7.8]])

In [792]:
A = 

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [823]:
X / np.sqrt(p)

array([[0.57735027, 2.88675135, 5.19615242],
       [1.15470054, 3.46410162, 5.77350269],
       [1.73205081, 4.04145188, 6.35085296],
       [2.30940108, 4.61880215, 6.92820323]])

In [941]:
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

In [916]:
gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2

(6, 0, 1, 1, array([0.60653066, 0.        , 0.        , 0.60653066, 0.        ,
        0.60653066]), array([0.00033546, 0.        , 0.00033546]), array([[5.2, 5.2, 5.4, 5.2, 5.4, 5.6],
        [6.2, 6.2, 6.4, 6.2, 6.4, 6.6],
        [7.2, 7.2, 7.4, 7.2, 7.4, 7.6]]), array([[5.4, 5.6, 5.6, 5.8, 5.8, 5.8],
        [6.4, 6.6, 6.6, 6.8, 6.8, 6.8],
        [7.4, 7.6, 7.6, 7.8, 7.8, 7.8]]), array([[5.2, 5.2, 6.2],
        [5.4, 5.4, 6.4],
        [5.6, 5.6, 6.6],
        [5.8, 5.8, 6.8]]), array([[6.2, 7.2, 7.2],
        [6.4, 7.4, 7.4],
        [6.6, 7.6, 7.6],
        [6.8, 7.8, 7.8]]))

In [917]:
w_l

array([0.60653066, 0.        , 0.        , 0.60653066, 0.        ,
       0.60653066])

In [942]:
vz = update_vz(lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2)

In [943]:
lambda_1, lambda_2 = lambdas[0], lambdas[1]
lambda_1

array([[-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ]])

In [929]:
w = np.array(dist_weight(X / np.sqrt(p),phi = 0.5))
w_l = knn_weights(w,1,n)
u = np.array(dist_weight(X.T / np.sqrt(n),phi = 0.5))
u_k = knn_weights(u,1,p)

In [953]:
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])
vz = update_vz(lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2)
v, z = vz[0], vz[1]
v

array([[-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ]])

In [954]:
z

array([[-1., -2., -1.],
       [-1., -2., -1.],
       [-1., -2., -1.],
       [-1., -2., -1.]])

In [955]:
nu2

1

In [815]:
    # update vz
sigma_1 = gamma_1 * w_l/nu1
vtemp = al1 - al2 - 1/nu1 * lambda_1
    
 #   temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
  #  temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 


In [819]:
sigma_1/np.sum(vtemp**2,axis=0)
np.sum(vtemp**2,axis=0)

array([0.12, 0.48, 0.12, 1.08, 0.48, 0.12])

In [822]:
w

array([0.60653066, 0.        , 0.        , 0.60653066, 0.        ,
       0.        ])

In [811]:
al1 - al2 

array([[-0.2, -0.4, -0.2, -0.6, -0.4, -0.2],
       [-0.2, -0.4, -0.2, -0.6, -0.4, -0.2],
       [-0.2, -0.4, -0.2, -0.6, -0.4, -0.2]])